In [1]:
import numpy as np
import pickle
import pandas as pd
from os.path import join
import warnings
import json
warnings.filterwarnings("ignore")
from sklearn.metrics import r2_score
from scipy import stats
import xgboost as xgb
from hyperopt import fmin, rand, hp, Trials


## Script to train and validate the gradient boosting models

Load training and test data

In [2]:
data_train = pd.read_pickle(join("partitions", "train_df.pkl"))
data_test = pd.read_pickle(join("partitions", "test_df.pkl"))

data_train = data_train.rename(columns={"kcat_km": "log10_kcat_km"})
data_test = data_test.rename(columns={"kcat_km": "log10_kcat_km"})

len(data_train), len(data_test)

(2375, 561)

In [4]:
train_indices = list(pd.read_pickle(join("partitions", "CV_train_indices.pkl")))
test_indices = list(pd.read_pickle(join("partitions", "CV_test_indices.pkl")))

### Training a model with only enzyme representations (ESM-1b)

Create input matrices

In [5]:
train_ESM1b = np.array(list(data_train["ESM1b"]))
train_X = train_ESM1b
train_Y = np.array(list(data_train["log10_kcat_km"]))

test_ESM1b = np.array(list(data_test["ESM1b"]))
test_X = test_ESM1b
test_Y = np.array(list(data_test["log10_kcat_km"]))

Hyperparameter optimization

In [ ]:
def cross_validation_mse_gradient_boosting(param):
    num_round = param["num_rounds"]
    del param["num_rounds"]
    param["max_depth"] = int(np.round(param["max_depth"]))
    param["tree_method"] = "gpu_hist"
    param["sampling_method"] = "gradient_based"
    
    MSE = []
    R2 = []
    for i in range(5):
        train_index, test_index  = train_indices[i], test_indices[i]
        dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
        dvalid = xgb.DMatrix(train_X[test_index])
        bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)
        y_valid_pred = bst.predict(dvalid)
        MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
        R2.append(r2_score(np.reshape(train_Y[test_index], (-1)),  y_valid_pred))
    return(-np.mean(R2))


space_gradient_boosting = {
    "learning_rate": hp.uniform("learning_rate", 0.01, 1),
    "max_depth": hp.uniform("max_depth", 4,12),
    #"subsample": hp.uniform("subsample", 0.7, 1),
    "reg_lambda": hp.uniform("reg_lambda", 0, 5),
    "reg_alpha": hp.uniform("reg_alpha", 0, 5),
    "max_delta_step": hp.uniform("max_delta_step", 0, 5),
    "min_child_weight": hp.uniform("min_child_weight", 0.1, 15),
    "num_rounds":  hp.uniform("num_rounds", 20, 200)}


trials = Trials()
best = fmin(fn = cross_validation_mse_gradient_boosting, space = space_gradient_boosting,
            algo=rand.suggest, max_evals = 200, trials=trials)

Training and validating model

In [23]:
param = best 

with open(join("/gpfs/project/dit55hov/best_param_ESM1b.json"), "w", encoding="utf-8") as f:
    json.dump(best, f, ensure_ascii=False, indent=4)

num_round = param["num_rounds"]
param["max_depth"] = int(np.round(param["max_depth"]))

del param["num_rounds"]

In [ ]:
R2 = []
MSE = []
Pearson = []

for i in range(5):
    train_index, test_index  = train_indices[i], test_indices[i]
    dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
    dvalid = xgb.DMatrix(train_X[test_index])
    
    bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)
    
    y_valid_pred = bst.predict(dvalid)
    MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
    R2.append(r2_score(np.reshape(train_Y[test_index], (-1)), y_valid_pred))
    Pearson.append(stats.pearsonr(np.reshape(train_Y[test_index], (-1)), y_valid_pred)[0])

print(Pearson)
print(MSE)
print(R2)

np.save(join("training_results", "Pearson_CV_xgboost_ESM1b.npy"), np.array(Pearson))
np.save(join("training_results", "MSE_CV_xgboost_ESM1b.npy"), np.array(MSE))
np.save(join("training_results", "R2_CV_xgboost_ESM1b.npy"), np.array(R2))

In [ ]:
dtrain = xgb.DMatrix(train_X, label = train_Y)
dtest = xgb.DMatrix(test_X)

bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

y_test_pred = bst.predict(dtest)
MSE_dif_fp_test = np.mean(abs(np.reshape(test_Y, (-1)) - y_test_pred)**2)
R2_dif_fp_test = r2_score(np.reshape(test_Y, (-1)), y_test_pred)
Pearson = stats.pearsonr(np.reshape(test_Y, (-1)), y_test_pred)

print(np.round(Pearson[0],3) ,np.round(MSE_dif_fp_test,3), np.round(R2_dif_fp_test,3))

np.save(join("training_results", "y_test_pred_xgboost_ESM1b.npy"), bst.predict(dtest))
np.save(join("training_results", "y_test_true_xgboost_ESM1b.npy"), test_Y)

### Training a model with reaction fingerprints (rxnfp)

Create input matrices

In [7]:
train_rxnfp = np.array(list(data_train["rxnfp"]))
train_X = train_rxnfp
train_Y = np.array(list(data_train["log10_kcat_km"]))

test_rxnfp = np.array(list(data_test["rxnfp"]))
test_X = test_rxnfp
test_Y = np.array(list(data_test["log10_kcat_km"]))

Hyperparameter optimization

In [ ]:
trials = Trials()
best = fmin(fn = cross_validation_mse_gradient_boosting, space = space_gradient_boosting,
            algo=rand.suggest, max_evals = 200, trials=trials)

In [ ]:
param = best 

with open(join("/gpfs/project/dit55hov/best_param_rxnfp.json"), "w", encoding="utf-8") as f:
    json.dump(best, f, ensure_ascii=False, indent=4)

num_round = param["num_rounds"]
param["max_depth"] = int(np.round(param["max_depth"]))

del param["num_rounds"]

In [ ]:
R2 = []
MSE = []
Pearson = []

for i in range(5):
    train_index, test_index  = train_indices[i], test_indices[i]
    dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
    dvalid = xgb.DMatrix(train_X[test_index])
    
    bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)
    
    y_valid_pred = bst.predict(dvalid)
    MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
    R2.append(r2_score(np.reshape(train_Y[test_index], (-1)), y_valid_pred))
    Pearson.append(stats.pearsonr(np.reshape(train_Y[test_index], (-1)), y_valid_pred)[0])

print(Pearson)
print(MSE)
print(R2)

np.save(join("training_results", "Pearson_CV_xgboost_rxnfp.npy"), np.array(Pearson))
np.save(join("training_results", "MSE_CV_xgboost_rxnfp.npy"), np.array(MSE))
np.save(join("training_results", "R2_CV_xgboost_rxnfp.npy"), np.array(R2))

In [ ]:
dtrain = xgb.DMatrix(train_X, label = train_Y)
dtest = xgb.DMatrix(test_X)

bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

y_test_pred = bst.predict(dtest)
MSE_dif_fp_test = np.mean(abs(np.reshape(test_Y, (-1)) - y_test_pred)**2)
R2_dif_fp_test = r2_score(np.reshape(test_Y, (-1)), y_test_pred)
Pearson = stats.pearsonr(np.reshape(test_Y, (-1)), y_test_pred)

print(np.round(Pearson[0],3) ,np.round(MSE_dif_fp_test,3), np.round(R2_dif_fp_test,3))

np.save(join("training_results", "y_test_pred_xgboost_rxnfp.npy"), bst.predict(dtest))
np.save(join("training_results", "y_test_true_xgboost_rxnfp.npy"), test_Y)

### Training a model with enzyme representations (ESM-1b) + reaction fingerprints (rxnfp)

Create input matrices

In [ ]:
train_rxnfp = np.array(list(data_train["esm1b_rxnfp"]))
train_X = train_rxnfp
train_Y = np.array(list(data_train["log10_kcat_km"]))

test_rxnfp = np.array(list(data_test["esm1b_rxnfp"]))
test_X = test_rxnfp
test_Y = np.array(list(data_test["log10_kcat_km"]))

Hyperparameter optimization

In [ ]:
trials = Trials()
best = fmin(fn = cross_validation_mse_gradient_boosting, space = space_gradient_boosting,
            algo=rand.suggest, max_evals = 200, trials=trials)

Training and validating model

In [ ]:
param = best

with open(join("/gpfs/project/dit55hov/best_param_esm1b_rxnfp.json"), "w", encoding="utf-8") as f:
    json.dump(best, f, ensure_ascii=False, indent=4)

num_round = param["num_rounds"]
param["max_depth"] = int(np.round(param["max_depth"]))

del param["num_rounds"]

In [ ]:
R2 = []
MSE = []
Pearson = []

for i in range(5):
    train_index, test_index  = train_indices[i], test_indices[i]
    dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
    dvalid = xgb.DMatrix(train_X[test_index])
    
    bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)
    
    y_valid_pred = bst.predict(dvalid)
    MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
    R2.append(r2_score(np.reshape(train_Y[test_index], (-1)), y_valid_pred))
    Pearson.append(stats.pearsonr(np.reshape(train_Y[test_index], (-1)), y_valid_pred)[0])

print(Pearson)
print(MSE)
print(R2)

np.save(join("training_results", "Pearson_CV_xgboost_esm1b_rxnfp.npy"), np.array(Pearson))
np.save(join("training_results", "MSE_CV_xgboost_esm1b_rxnfp.npy"), np.array(MSE))
np.save(join("training_results", "R2_CV_xgboost_esm1b_rxnfp.npy"), np.array(R2))

In [ ]:
dtrain = xgb.DMatrix(train_X, label = train_Y)
dtest = xgb.DMatrix(test_X)

bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

y_test_pred = bst.predict(dtest)
MSE_dif_fp_test = np.mean(abs(np.reshape(test_Y, (-1)) - y_test_pred)**2)
R2_dif_fp_test = r2_score(np.reshape(test_Y, (-1)), y_test_pred)
Pearson = stats.pearsonr(np.reshape(test_Y, (-1)), y_test_pred)

print(np.round(Pearson[0],3) ,np.round(MSE_dif_fp_test,3), np.round(R2_dif_fp_test,3))

np.save(join("training_results", "y_test_pred_xgboost_esm1b_rxnfp.npy"), bst.predict(dtest))
np.save(join("training_results", "y_test_true_xgboost_esm1b_rxnfp.npy"), test_Y)

### Training a model with only SMILES substrate representations (ChemBERTa2)

Create input matrices

In [ ]:
train_ChemBERTa2 = np.array(list(data_train["ChemBERTa2"]))
train_X = train_ChemBERTa2
train_Y = np.array(list(data_train["log10_kcat_km"]))

test_ChemBERTa2 = np.array(list(data_test["ChemBERTa2"]))
test_X = test_ChemBERTa2
test_Y = np.array(list(data_test["log10_kcat_km"]))

Hyperparameter optimization

In [ ]:
trials = Trials()
best = fmin(fn = cross_validation_mse_gradient_boosting, space = space_gradient_boosting,
            algo=rand.suggest, max_evals = 200, trials=trials)

Training and validating model

In [ ]:
param = best 

with open(join("/gpfs/project/dit55hov/best_param_ESM1b.json"), "w", encoding="utf-8") as f:
    json.dump(best, f, ensure_ascii=False, indent=4)

num_round = param["num_rounds"]
param["max_depth"] = int(np.round(param["max_depth"]))

del param["num_rounds"]

In [ ]:
R2 = []
MSE = []
Pearson = []

for i in range(5):
    train_index, test_index  = train_indices[i], test_indices[i]
    dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
    dvalid = xgb.DMatrix(train_X[test_index])
    
    bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)
    
    y_valid_pred = bst.predict(dvalid)
    MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
    R2.append(r2_score(np.reshape(train_Y[test_index], (-1)), y_valid_pred))
    Pearson.append(stats.pearsonr(np.reshape(train_Y[test_index], (-1)), y_valid_pred)[0])

print(Pearson)
print(MSE)
print(R2)

np.save(join("training_results", "Pearson_CV_xgboost_ChemBERTa2.npy"), np.array(Pearson))
np.save(join("training_results", "MSE_CV_xgboost_ChemBERTa2.npy"), np.array(MSE))
np.save(join("training_results", "R2_CV_xgboost_ChemBERTa2.npy"), np.array(R2))

In [ ]:
dtrain = xgb.DMatrix(train_X, label = train_Y)
dtest = xgb.DMatrix(test_X)

bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

y_test_pred = bst.predict(dtest)
MSE_dif_fp_test = np.mean(abs(np.reshape(test_Y, (-1)) - y_test_pred)**2)
R2_dif_fp_test = r2_score(np.reshape(test_Y, (-1)), y_test_pred)
Pearson = stats.pearsonr(np.reshape(test_Y, (-1)), y_test_pred)

print(np.round(Pearson[0],3) ,np.round(MSE_dif_fp_test,3), np.round(R2_dif_fp_test,3))

np.save(join("training_results", "y_test_pred_xgboost_ChemBERTa2.npy"), bst.predict(dtest))
np.save(join("training_results", "y_test_true_xgboost_ChemBERTa2.npy"), test_Y)